In [ ]:
from pathlib import Path
import base64
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto import Random
from tqdm import tqdm
from getpass import getpass

In [ ]:
def encrypt(key, source, encode=True):
    key = SHA256.new(
        key
    ).digest()  # use SHA-256 over our key to get a proper-sized AES key
    IV = Random.new().read(AES.block_size)  # generate IV
    encryptor = AES.new(key, AES.MODE_CBC, IV)
    padding = AES.block_size - len(source) % AES.block_size  # calculate needed padding
    source += bytes([padding]) * padding  # Python 2.x: source += chr(padding) * padding
    data = IV + encryptor.encrypt(source)  # store the IV at the beginning and encrypt
    return base64.b64encode(data).decode("latin-1") if encode else data

In [ ]:
def decrypt(key, source, decode=True):
    if decode:
        source = base64.b64decode(source.encode("latin-1"))
    key = SHA256.new(
        key
    ).digest()  # use SHA-256 over our key to get a proper-sized AES key
    IV = source[: AES.block_size]  # extract the IV from the beginning
    decryptor = AES.new(key, AES.MODE_CBC, IV)
    data = decryptor.decrypt(source[AES.block_size:])  # decrypt
    padding = data[-1]  # pick the padding value from the end; Python 2.x: ord(data[-1])
    if (
        data[-padding:] != bytes([padding]) * padding
    ):  # Python 2.x: chr(padding) * padding
        raise ValueError("Invalid padding...")
    return data[:-padding]  # remove the padding

In [ ]:
def decryptFile(path,my_password):
    encrypted = path.read_bytes()
    decrypted = decrypt(my_password, encrypted, decode=False)
    path.write_bytes(decrypted)

In [ ]:
def encryptFilename(path,my_password):
    new_name = path.name.encode('utf-8')
    new_name = encrypt(my_password, new_name, encode=False)
    new_name = base64.b64encode(new_name)
    new_name = new_name.replace(b'+', b'-').replace(b'/', b'_')
    new_name = new_name.decode('utf-8')
    path.rename(path.with_name(new_name).with_suffix(".enc"))

def decryptFilename(path,my_password):
    new_name = path.stem.encode('utf-8')
    new_name = new_name.replace(b'-', b'+').replace(b'_', b'/')
    new_name = base64.b64decode(new_name)
    new_name = decrypt(my_password, new_name,decode=False)
    new_name = new_name.decode('utf-8')
    path.rename(path.with_name(new_name))

In [ ]:
def encryptFile(path, my_password):
    my_data = path.read_bytes()
    encrypted = encrypt(my_password, my_data, encode=False)
    path.write_bytes(encrypted)

In [ ]:
password =getpass().encode('utf-8')
path = Path('/media/srudloff/7CD6-1313/enc')

In [ ]:
if input("Encrypt? Y/n").lower() == 'y':
    for file in tqdm(path.glob('**/*')):
        if file.is_dir():
            if not file.name.endswith('.enc'):
                encryptFilename(file,password)
    for file in tqdm(path.glob('**/*')):
        if file.is_file():
            if file.suffix != '.enc':
                encryptFile(file, password)
                encryptFilename(file, password)

In [ ]:
if input("Decrypt? Y/n").lower() == 'y':
    for file in tqdm(path.glob('**/*')):
        if file.is_dir():
            if file.name.endswith('.enc'):
                decryptFilename(file, password)

    for file in tqdm(path.glob('**/*')):
        if file.is_file():
            if file.suffix == '.enc':
                try:
                    print(file, end ="\t")
                    decryptFile(file, password)
                    decryptFilename(file, password)
                    print("decrypted")
                except ValueError:
                    print("Bad Password")
                